In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, roc_auc_score, confusion_matrix, \
    ConfusionMatrixDisplay, balanced_accuracy_score, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import itertools
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn import metrics


In [7]:
all_train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)


In [8]:
all_train.head()

,user,track,time,user_avg_time,user_track_count,track_avg_time,track_listen_count
0,ee8621197232afef4ae573079d64480ba7640c9eb91a7e...,41378,1.0,0.169500,180,0.214737,38
1,9b19f9ab816598a0809e4afd5d60800f2dbef9cbb9b03a...,44158,0.0,0.148226,124,0.182069,29
2,be3d629f02589a093027c0b917aa7668f22b2f89e83328...,1263,0.8,0.201677,167,0.523857,70
3,3d810b358ef3f88619945df4182acda92a24e99c8b28e8...,3781,0.0,0.262101,138,0.201351,37
4,2261b761b06d13e4ca4118d58d91eb7312ae864b2916d7...,32711,0.0,0.189217,166,0.173600,25


In [9]:
test

,user,track,user_avg_time,user_track_count,track_avg_time,track_listen_count
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,4218,0.206739,92,0.482623,61
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,19007,0.206739,92,0.175429,35
2,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,346,0.206739,92,0.209697,33
3,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,8822,0.206739,92,0.137083,24
4,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,29271,0.206739,92,0.111463,41
...,...,...,...,...,...,...
43255,888df25ae35772424a560c7152a1de794440e0ea5cfee6...,36867,0.137009,117,0.183939,33
43256,888df25ae35772424a560c7152a1de794440e0ea5cfee6...,5948,0.137009,117,0.228485,33
43257,888df25ae35772424a560c7152a1de794440e0ea5cfee6...,46988,0.137009,117,0.104444,18
43258,888df25ae35772424a560c7152a1de794440e0ea5cfee6...,18083,0.137009,117,0.163000,30


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(all_train['track'], all_train['time'], test_size=0.2, random_state=52)

In [11]:
features = ['user_avg_time', 'user_track_count', 'track_avg_time', 'track_listen_count']
target = 'time'

In [19]:
param_dist = {
    'max_depth': randint(3, 10),  # глубина деревьев
    'learning_rate': uniform(0.01, 0.3),  # скорость обучения
    'n_estimators': randint(100, 1000),  # количество деревьев
    'subsample': uniform(0.5, 0.5),  # доля наблюдений для построения одного дерева
    'colsample_bytree': uniform(0.5, 0.5),  # доля признаков для построения одного дерева
}
xgb_model = XGBRegressor(objective='rank:ndcg')

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=1,  # количество итераций случайного поиска
    cv=5,  # количество разбиений для кросс-валидации
    scoring='neg_mean_squared_error',  # метрика для оценки качества модели
    random_state=52,
    verbose=4
)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [67]:
def dcg_at_k(r, k):
    """Вычисление Discounted Cumulative Gain (DCG) для списка оценок r до позиции k."""
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k):
    """Вычисление Normalized Discounted Cumulative Gain (NDCG) для списка оценок r до позиции k."""
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [ ]:
ndcg_at_k()